In [1]:
import glob
import sys
import math
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import tensorflow as tf

sys.path.append("../../")

from helpers.experiment import experiment
from helpers.assess import resolve_sample_weight
from helpers.split import make_train_test_split, tag_label_feature_split
import matplotlib.pyplot as plt
import math
from helpers.keras_graph_callback import KerasGraphCallback
from sklearn.utils.class_weight import compute_class_weight

/usr/local/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_pickle('../../datasets/dataset_10_melspecs.pickle')
df = df.sample(frac=1) #shuffle

In [6]:
_, (y, le), X = tag_label_feature_split(
    df, label_format="encoded"
)

In [7]:
# extract 2D numpy array from pandas dataframe
X = np.array(list(X.to_numpy()[:,0]))
# choosing ~5s clip to make square 96x96 spectrogram frame
X = X[:,0:96,200:296]

scaler = StandardScaler()
# flatten 2D array to fit to "overall" mean / variance
scaler.fit(X.reshape(-1,1))
# must be reshaped for transformation then restored to original shape
X = scaler.transform(X.reshape(-1, 1)).reshape(X.shape)

In [8]:
X_train, X_validation, y_train, y_validation = make_train_test_split(
        X, y, test_size=0.3, random_state=10, stratify=y
    )
class_weight={i:c  for i,c in enumerate(compute_class_weight(class_weight='balanced',classes=le.transform(le.classes_),y=y_train))}

In [61]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, 16, activation='relu', input_shape=(96, 96,1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, 8, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, 4, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
# model.add(tf.keras.layers.Conv2D(16, 4, activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(10))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history = model.fit(
    X_train,
    y_train,
    epochs=200, 
    validation_data=(X_validation,y_validation),
    class_weight=class_weight
)

Epoch 1/200
 62/468 [==>...........................] - ETA: 4:39 - loss: 2.6184 - accuracy: 0.0827

KeyboardInterrupt: 